In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/pggan/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 7000
start_epoch = 0
train_batch = 12
test_batch = 200
lr = 0.03
schedule = [200, 1500, 3000, 4500, 6000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b1/to_style1/cutmix3' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = 'fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.1

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(target_dir, '100_shot_pggan_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        source_inputs, source_targets = source_set[batch_idx]
        source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        st = source_targets[rand_index]
        tt = targets[rand_index]
        rand_index = rand_index[st == tt]

        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), 1-lam)
        inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 20 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 7000] LR: 0.030000
1/17 Data:1.410 | Batch:5.780 | Total:0:00:05 | ETA:0:01:22 | Loss:1.6891463994979858 | top1:83.33332824707031
2/17 Data:0.003 | Batch:0.977 | Total:0:00:06 | ETA:0:00:46 | Loss:1.2112552225589752 | top1:79.16667175292969
3/17 Data:0.001 | Batch:0.797 | Total:0:00:06 | ETA:0:00:32 | Loss:1.5493227044741313 | top1:72.22222137451172
4/17 Data:0.002 | Batch:0.869 | Total:0:00:07 | ETA:0:00:26 | Loss:1.9355444461107254 | top1:64.58333587646484
5/17 Data:0.001 | Batch:0.693 | Total:0:00:08 | ETA:0:00:21 | Loss:1.7239863753318787 | top1:65.0
6/17 Data:0.002 | Batch:0.569 | Total:0:00:08 | ETA:0:00:17 | Loss:1.5947136183579762 | top1:62.5
7/17 Data:0.002 | Batch:0.616 | Total:0:00:09 | ETA:0:00:14 | Loss:1.6034826125417436 | top1:65.47618865966797
8/17 Data:0.003 | Batch:0.830 | Total:0:00:10 | ETA:0:00:12 | Loss:1.650319628417492 | top1:63.54166793823242
9/17 Data:0.002 | Batch:0.949 | Total:0:00:11 | ETA:0:00:11 | Loss:1.6428895195325215 | top1:66.66666412353

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


39/39 Data:0.012 | Batch:0.337 | Total:0:00:32 | ETA:0:00:00 | Loss:1.8405131590672028 | top1:52.17948532104492
161/161 Data:0.013 | Batch:1.358 | Total:0:01:03 | ETA:0:00:00 | Loss:1.3214797955064388 | top1:96.57009887695312

Epoch: [2 | 7000] LR: 0.031800
1/17 Data:0.801 | Batch:1.218 | Total:0:00:00 | ETA:0:00:14 | Loss:1.8860352039337158 | top1:50.0
2/17 Data:0.011 | Batch:0.347 | Total:0:00:01 | ETA:0:00:10 | Loss:1.8190870881080627 | top1:58.333335876464844
3/17 Data:0.002 | Batch:0.491 | Total:0:00:01 | ETA:0:00:08 | Loss:1.7511516014734905 | top1:66.66666412353516
4/17 Data:0.002 | Batch:0.699 | Total:0:00:02 | ETA:0:00:08 | Loss:1.7185599207878113 | top1:68.75
5/17 Data:0.003 | Batch:0.550 | Total:0:00:02 | ETA:0:00:08 | Loss:1.6938888072967528 | top1:68.33333587646484
6/17 Data:0.005 | Batch:0.313 | Total:0:00:03 | ETA:0:00:06 | Loss:1.6978830099105835 | top1:65.27777862548828
7/17 Data:0.003 | Batch:0.412 | Total:0:00:03 | ETA:0:00:06 | Loss:1.6522597074508667 | top1:69.0476

9/17 Data:0.002 | Batch:0.276 | Total:0:00:04 | ETA:0:00:05 | Loss:1.2205539279513888 | top1:78.70370483398438
10/17 Data:0.032 | Batch:0.433 | Total:0:00:04 | ETA:0:00:04 | Loss:1.210348904132843 | top1:79.16667175292969
11/17 Data:0.003 | Batch:0.394 | Total:0:00:05 | ETA:0:00:03 | Loss:1.227604248306968 | top1:78.03030395507812
12/17 Data:0.007 | Batch:0.321 | Total:0:00:05 | ETA:0:00:03 | Loss:1.219701995452245 | top1:77.77777862548828
13/17 Data:0.015 | Batch:0.606 | Total:0:00:06 | ETA:0:00:02 | Loss:1.2245690822601318 | top1:77.56410217285156
14/17 Data:0.004 | Batch:0.667 | Total:0:00:06 | ETA:0:00:02 | Loss:1.228954247065953 | top1:77.97618865966797
15/17 Data:0.003 | Batch:0.584 | Total:0:00:07 | ETA:0:00:01 | Loss:1.2315383593241374 | top1:77.77777862548828
16/17 Data:0.001 | Batch:0.377 | Total:0:00:07 | ETA:0:00:01 | Loss:1.229864925146103 | top1:77.08333587646484

Epoch: [7 | 7000] LR: 0.040800
1/17 Data:1.070 | Batch:1.613 | Total:0:00:01 | ETA:0:00:18 | Loss:1.262552261

3/17 Data:0.002 | Batch:2.671 | Total:0:00:13 | ETA:0:01:03 | Loss:1.122814377148946 | top1:75.0
4/17 Data:0.001 | Batch:2.661 | Total:0:00:15 | ETA:0:00:52 | Loss:1.0523956418037415 | top1:75.0
5/17 Data:0.001 | Batch:1.834 | Total:0:00:17 | ETA:0:00:43 | Loss:0.9886074781417846 | top1:80.00000762939453
6/17 Data:0.003 | Batch:2.169 | Total:0:00:19 | ETA:0:00:37 | Loss:0.9368242025375366 | top1:83.33333587646484
7/17 Data:0.052 | Batch:2.132 | Total:0:00:22 | ETA:0:00:32 | Loss:0.9342503377369472 | top1:82.14286041259766
8/17 Data:0.001 | Batch:2.244 | Total:0:00:24 | ETA:0:00:28 | Loss:0.9127263501286507 | top1:82.29167175292969
9/17 Data:0.001 | Batch:1.566 | Total:0:00:25 | ETA:0:00:24 | Loss:0.9073917667071024 | top1:82.40740966796875
10/17 Data:0.001 | Batch:1.951 | Total:0:00:27 | ETA:0:00:20 | Loss:0.9148016035556793 | top1:81.66667175292969
11/17 Data:0.001 | Batch:2.350 | Total:0:00:30 | ETA:0:00:14 | Loss:0.9179527922110124 | top1:81.06060791015625
12/17 Data:0.001 | Batch:1

...

16/17 Data:0.001 | Batch:2.915 | Total:0:00:56 | ETA:0:00:04 | Loss:0.6968568935990334 | top1:93.22917175292969

Epoch: [760 | 7000] LR: 0.011700
1/17 Data:4.389 | Batch:6.475 | Total:0:00:03 | ETA:0:00:58 | Loss:0.5583459138870239 | top1:100.0
2/17 Data:0.001 | Batch:1.941 | Total:0:00:05 | ETA:0:00:42 | Loss:0.5515038967132568 | top1:100.0
3/17 Data:0.001 | Batch:2.352 | Total:0:00:07 | ETA:0:00:37 | Loss:0.5704769492149353 | top1:97.22222137451172
4/17 Data:0.003 | Batch:2.564 | Total:0:00:10 | ETA:0:00:35 | Loss:0.6310025304555893 | top1:93.75
5/17 Data:0.001 | Batch:3.304 | Total:0:00:13 | ETA:0:00:34 | Loss:0.6885120511054993 | top1:91.66667175292969
6/17 Data:0.001 | Batch:4.072 | Total:0:00:17 | ETA:0:00:33 | Loss:0.7026606599489847 | top1:91.66666412353516
7/17 Data:0.001 | Batch:4.127 | Total:0:00:22 | ETA:0:00:32 | Loss:0.7085334403174264 | top1:90.47618865966797
8/17 Data:0.048 | Batch:4.032 | Total:0:00:26 | ETA:0:00:30 | Loss:0.7154461517930031 | top1:89.58333587646484
9/

8/17 Data:0.001 | Batch:2.501 | Total:0:00:34 | ETA:0:00:39 | Loss:0.5985197871923447 | top1:95.83333587646484
9/17 Data:0.001 | Batch:3.430 | Total:0:00:37 | ETA:0:00:34 | Loss:0.5913375748528374 | top1:96.29629516601562
10/17 Data:0.002 | Batch:2.638 | Total:0:00:40 | ETA:0:00:29 | Loss:0.5977779030799866 | top1:95.83333587646484
11/17 Data:0.000 | Batch:3.249 | Total:0:00:43 | ETA:0:00:22 | Loss:0.5898512385108254 | top1:96.21212768554688
12/17 Data:0.002 | Batch:3.403 | Total:0:00:47 | ETA:0:00:19 | Loss:0.5839362889528275 | top1:96.52777862548828
13/17 Data:0.000 | Batch:3.571 | Total:0:00:50 | ETA:0:00:15 | Loss:0.5802651047706604 | top1:96.79487609863281
14/17 Data:0.001 | Batch:4.532 | Total:0:00:55 | ETA:0:00:11 | Loss:0.5803331434726715 | top1:96.42857360839844
15/17 Data:0.027 | Batch:3.433 | Total:0:00:58 | ETA:0:00:08 | Loss:0.5741931696732839 | top1:96.66667175292969
16/17 Data:0.000 | Batch:3.776 | Total:0:01:02 | ETA:0:00:04 | Loss:0.5688210092484951 | top1:96.875

Epoc

2/17 Data:0.024 | Batch:3.074 | Total:0:00:06 | ETA:0:00:50 | Loss:0.4440314322710037 | top1:100.0
3/17 Data:0.003 | Batch:2.935 | Total:0:00:09 | ETA:0:00:45 | Loss:0.4463404913743337 | top1:100.0
4/17 Data:0.005 | Batch:2.523 | Total:0:00:12 | ETA:0:00:40 | Loss:0.45337333530187607 | top1:100.0
5/17 Data:0.001 | Batch:0.348 | Total:0:00:12 | ETA:0:00:30 | Loss:0.4524649024009705 | top1:100.00000762939453
6/17 Data:0.018 | Batch:0.388 | Total:0:00:12 | ETA:0:00:24 | Loss:0.4625121255715688 | top1:100.0
7/17 Data:0.016 | Batch:0.446 | Total:0:00:13 | ETA:0:00:19 | Loss:0.460011967590877 | top1:100.0
8/17 Data:0.001 | Batch:0.786 | Total:0:00:14 | ETA:0:00:16 | Loss:0.4617825895547867 | top1:100.0
9/17 Data:0.004 | Batch:2.416 | Total:0:00:16 | ETA:0:00:15 | Loss:0.4588004880481296 | top1:100.0
10/17 Data:0.002 | Batch:3.869 | Total:0:00:20 | ETA:0:00:15 | Loss:0.4616667240858078 | top1:100.00000762939453
11/17 Data:0.001 | Batch:3.276 | Total:0:00:23 | ETA:0:00:13 | Loss:0.467739850282

12/17 Data:0.001 | Batch:0.288 | Total:0:00:11 | ETA:0:00:06 | Loss:0.5715399334828059 | top1:91.66666412353516
13/17 Data:0.003 | Batch:0.241 | Total:0:00:11 | ETA:0:00:05 | Loss:0.5995982747811538 | top1:90.38461303710938
14/17 Data:0.022 | Batch:0.273 | Total:0:00:11 | ETA:0:00:01 | Loss:0.6047407771859851 | top1:90.47618865966797
15/17 Data:0.013 | Batch:0.263 | Total:0:00:11 | ETA:0:00:01 | Loss:0.5930709679921468 | top1:91.11111450195312
16/17 Data:0.025 | Batch:0.273 | Total:0:00:12 | ETA:0:00:01 | Loss:0.595226027071476 | top1:91.14583587646484

Epoch: [774 | 7000] LR: 0.011688
1/17 Data:0.682 | Batch:1.090 | Total:0:00:00 | ETA:0:00:12 | Loss:0.4571261405944824 | top1:100.0
2/17 Data:0.015 | Batch:0.376 | Total:0:00:01 | ETA:0:00:09 | Loss:0.5210038423538208 | top1:95.83333587646484
3/17 Data:0.016 | Batch:0.372 | Total:0:00:01 | ETA:0:00:07 | Loss:0.5143202940622965 | top1:94.44444274902344
4/17 Data:0.036 | Batch:0.397 | Total:0:00:01 | ETA:0:00:07 | Loss:0.5262941122055054 

6/17 Data:0.005 | Batch:1.253 | Total:0:00:25 | ETA:0:00:47 | Loss:0.46008413036664325 | top1:97.22222137451172
7/17 Data:0.009 | Batch:0.946 | Total:0:00:26 | ETA:0:00:38 | Loss:0.4488277392727988 | top1:97.61904907226562
8/17 Data:0.001 | Batch:4.276 | Total:0:00:30 | ETA:0:00:35 | Loss:0.44193554297089577 | top1:97.91667175292969
9/17 Data:0.001 | Batch:3.641 | Total:0:00:34 | ETA:0:00:31 | Loss:0.44219742218653363 | top1:98.14814758300781
10/17 Data:0.001 | Batch:3.841 | Total:0:00:38 | ETA:0:00:27 | Loss:0.44233905971050264 | top1:98.33333587646484
11/17 Data:0.001 | Batch:4.265 | Total:0:00:42 | ETA:0:00:20 | Loss:0.4394862814383073 | top1:98.48484802246094
12/17 Data:0.001 | Batch:4.809 | Total:0:00:47 | ETA:0:00:17 | Loss:0.4334758147597313 | top1:98.61111450195312
13/17 Data:0.001 | Batch:4.182 | Total:0:00:51 | ETA:0:00:13 | Loss:0.4323009711045485 | top1:98.71794891357422
14/17 Data:0.001 | Batch:4.029 | Total:0:00:55 | ETA:0:00:10 | Loss:0.42787826699869974 | top1:98.809524